In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from IPython.display import clear_output
import pandas as pd
import multiprocessing
from os import listdir
from os.path import isfile, join
import warnings
warnings.filterwarnings("ignore")
from multiprocessing.managers import BaseManager

pool = multiprocessing.Pool()

url = 'https://www.govinfo.gov/sitemap/BILLS_sitemap_index.xml'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text,'xml')
level_0=pd.read_xml(soup.prettify())
level_0['year']=(level_0['loc'].str.rsplit('BILLS_', expand=True)[1]).str.replace('_sitemap.xml', '')
level_0['year']=level_0['year'].replace(' ','')
level_0=level_0.sort_values(by='year', ascending=False).reset_index(drop=True)

# totalfile=0
# yearly_files=[]
# for i in level_0['loc'].tolist():
#     url2 = i
#     reqs2 = requests.get(url2)
#     soup2 = BeautifulSoup(reqs2.text,'xml')
#     level_1=pd.read_xml(soup2.prettify())
#     #only want bills introduced in house and introduced in senate
#     level_1=level_1[level_1['loc'].str.upper().str.contains("IS|IH")].reset_index(drop=True)
#     thisyr=len(level_1['loc'].tolist())
#     yearly_files.append(thisyr)
#     totalfile=totalfile+thisyr
#     print(totalfile)

def pull_year(yr, loc):
    for year, level_0_location in zip(yr, loc):
        url2 = level_0_location
        reqs2 = requests.get(url2)
        soup2 = BeautifulSoup(reqs2.text,'xml')
        level_1=pd.read_xml(soup2.prettify())
        level_1['placehold']=level_1['loc'].str.rsplit(n=1, pat='/', expand=True)[1]
        level_1['placehold']=level_1['placehold'].str.replace(' ','')
        level_1['year']=year
        level_1['name']=level_1['year']+level_1['placehold']
        level_1['url']=level_1['loc'].str.replace('app/details', 'content/pkg')+'/html/'+level_1['loc'].str.rsplit(n=1, pat='/', expand=True)[1]+'.htm'
        level_1=level_1[level_1['loc'].str.upper().str.contains("IS|IH")].reset_index(drop=True)
        #only want bills introduced in house and introduced in senate
        counter=0
        for document, doc_name in zip(level_1['url'].tolist(), level_1['name'].tolist()):
            complete=round((counter/totalfile),4)
            counter=counter+1
            r = requests.get('{}'.format(document), allow_redirects=True)
            if 'ih' in doc_name:
                open('C:/Users/micha/OneDrive/Desktop/453/bills/{}/House/{}.html'.format(year, doc_name), 'wb').write(r.content)
            else:
                open('C:/Users/micha/OneDrive/Desktop/453/bills/{}/Senate/{}.html'.format(year, doc_name), 'wb').write(r.content)
            complete=round((counter/totalfile)*100,4)
            print("Current Year: {}    Overall Percent Complete: {:7}%    Files Processed: {:8}    Files Remaining: {:8}".format(year, str(complete), counter,str(totalfile-counter)))
            counter=counter+1
            clear_output(wait=True)


multiprocessing.Pool(pull_year(level_0['year'].tolist(), level_0['loc']).tolist(), range(8))